In [1]:
import java.nio.file.Files
import java.nio.file.Path

data class BenchmarkData(val name: String, val score: Double, val error: Double) {
    override fun toString(): String = "$name: $score ± $error ns/op"
}

fun readHotspotBenchmarkData(benchmarkName: String) = Files.readAllLines(Path.of("build", "results", "jmh", "results.txt"))
    .mapNotNull {
        val r = Regex("$benchmarkName\\.([^ ]+) +avgt +\\d+ +([0-9.]+) +± +([0-9.]+) +ns/op")
        val match = r.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            match.groups[1]!!.value,
            match.groups[2]!!.value.toDouble(),
            match.groups[3]!!.value.toDouble(),
        )
    }
(readHotspotBenchmarkData("Wolf3dBenchmark") + readHotspotBenchmarkData("FibonacciBenchmark")).joinToString("\n")

baselineDouble: 3.4155478943E7 ± 159260.73 ns/op
baselineFloat: 3.3262500217E7 ± 163188.313 ns/op
inlineDouble: 3.4809097983E7 ± 204029.492 ns/op
inlineFloat: 3.2958106082E7 ± 201399.308 ns/op
longPackFloat: 5.6056574704E7 ± 57637.248 ns/op
mutableRefDouble: 4.0088944943E7 ± 129946.423 ns/op
mutableRefFloatIn1Long: 6.4626704545E7 ± 259191.875 ns/op
mutableRefFloatIn2Longs: 3.942654435E7 ± 158509.538 ns/op
valueDouble: 2.4692335152E7 ± 96987.297 ns/op
valueFloat: 2.4975946511E7 ± 42944.141 ns/op
valueInlineDouble: 2.4727345047E7 ± 162216.549 ns/op
valueInlineFloat: 2.5028369897E7 ± 32173.83 ns/op
valuePreserveBoxDouble: 2.461521608E7 ± 38902.754 ns/op
valuePreserveBoxFloat: 2.5072672279E7 ± 47553.798 ns/op

In [2]:
%use lets-plot

In [3]:
import org.jetbrains.letsPlot.intern.PosKind
import org.jetbrains.letsPlot.intern.StatKind
import org.jetbrains.letsPlot.intern.layer.PosOptions
import org.jetbrains.letsPlot.intern.layer.StatOptions

fun makePlot(name: String, platform: String, benchmarkLines: List<BenchmarkData>) {
    if (benchmarkLines.isEmpty()) return
    val names = benchmarkLines.map { it.name }
    val scores = benchmarkLines.map { it.score }
    val scoresMinusErrors = benchmarkLines.map { it.score - it.error / 2 }
    val scoresPlusErrors = benchmarkLines.map { it.score + it.error / 2 }
    val baseline = listOf("Float", "Double", "Int", "Long").mapNotNull { type ->
        val result = benchmarkLines.singleOrNull { it.name == "baseline$type" }
        if (result != null) type to result else null
    }.toMap()
    val maxScore = scoresPlusErrors.max()
    val plot = letsPlot() +
            labs("$name ($platform)", fill = "Benchmark", y = "ns/op") +
            geomBar(stat = StatOptions(kind = StatKind.IDENTITY)) {
                x = names
                y = scores
                fill = names
            } +
            geomErrorBar(
                width = 0.5,
                position = PosOptions(kind = PosKind.DODGE),
                size = 0.6
            ) {
                x = names
                ymin = scoresMinusErrors
                ymax = scoresPlusErrors
            } +
            geomText(
                stat = StatOptions(kind = StatKind.IDENTITY),
                position = positionStack(vjust = 1.0),
                size = 5,
            ) {
                x = names
                y = scores.map {
                    val threshold = 0.08
                    if (it < maxScore * threshold) it + threshold * maxScore / 2 else it / 2
                }
                label = benchmarkLines.map {
                    val type = when {
                        it.name.contains("Float") -> "Float"
                        it.name.contains("Int") -> "Int"
                        it.name.contains("Double") -> "Double"
                        it.name.contains("Long") -> "Long"
                        else -> error("Unknown name: ${it.name}")
                    }
                    val score = it.score / baseline[type]!!.score
                    val error = it.error / baseline[type]!!.score
                    "${String.format("%.0f", score * 100)}%\n±\n${String.format("%.1f", error * 100)}%\n${type}"
                }
            } +
            scaleXDiscrete(
                labels = names,
                name = "",
                limits = names + listOf(""),
                breaks = names.toList()
            ) +
            scaleYContinuous(limits = 0 to maxScore, name = "ns/op") +
            theme(plotTitle = elementText(hjust = 0.5)).legendPositionRight() +
            ggsize(1000, 800)
    ggsave(plot = plot, filename = "$name ($platform).svg")
    plot.show()
}


In [4]:
makePlot("Wolf3d", "Hotspot", readHotspotBenchmarkData("Wolf3dBenchmark"))
makePlot("Ackermann", "Hotspot", readHotspotBenchmarkData("AckermannBenchmark"))

In [5]:
import java.io.FileFilter
import kotlin.io.path.listDirectoryEntries

fun readArtBenchmarkData(benchmarkName: String) = Path.of("android-benchmark/build/outputs/androidTest-results/connected")
    .toFile().listFiles(FileFilter { it.isDirectory })!!.single()
    .listFiles { dir, name -> name.startsWith("logcat-org.jetbrains.") }!!.flatMap { it.readLines() }
    .mapNotNull {
        val pattern = Regex("^\\d{2}-\\d{2} +\\d+:\\d+:\\d+.\\d+ +\\d+ +\\d+ +I +Benchmark: +$benchmarkName\\.([^\\[]+)\\[Metric \\(timeNs\\) +results: +median +([^,]+), +min [^,]+, +max [^,]+, +standardDeviation: +([^,]+), +.*")
        val match = pattern.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            match.groups[1]!!.value,
            match.groups[2]!!.value.toDouble(),
            match.groups[3]!!.value.toDouble(),
        )
     }.let { lines ->
         val indexes = readHotspotBenchmarkData(benchmarkName).mapIndexed { index, benchmarkData -> benchmarkData.name to index }.toMap()
         if (lines.all { it.name in indexes }) lines.sortedBy { indexes[it.name]!! } else lines
     }
(readArtBenchmarkData("Wolf3dBenchmark") + readArtBenchmarkData("FibonacciBenchmark")).joinToString("\n")

baselineDouble: 3.83608346E8 ± 1.3389502088539738E7 ns/op
baselineFloat: 3.09443365E8 ± 6211590.393370372 ns/op
inlineDouble: 8.392054805E8 ± 2.3634348439076178E7 ns/op
inlineFloat: 6.72639326E8 ± 1.957563331133731E7 ns/op
longPackFloat: 1.17564808E8 ± 821793.8851736297 ns/op
mutableRefDouble: 1.06691538E8 ± 2431715.160789974 ns/op
mutableRefFloatIn1Long: 1.66585961E8 ± 2401852.2491979324 ns/op
mutableRefFloatIn2Longs: 1.019348075E8 ± 1280932.56373119 ns/op
valueDouble: 4.32730596E8 ± 7161589.385863689 ns/op
valueFloat: 2.760663845E8 ± 7439224.6356679695 ns/op
valueInlineDouble: 1.86463692E7 ± 524902.0075929541 ns/op
valueInlineFloat: 1.76072307E7 ± 17422.680620258012 ns/op
valuePreserveBoxDouble: 4.10934673E8 ± 1.3126733572686173E7 ns/op
valuePreserveBoxFloat: 2.97113942E8 ± 6873206.490616193 ns/op

In [6]:
makePlot("Wolf3d", "ART", readArtBenchmarkData("Wolf3dBenchmark"))
makePlot("Ackermann", "ART", readArtBenchmarkData("AckermannBenchmark"))